In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

In [4]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
X_train_full.shape

(60000, 28, 28, 1)

In [5]:
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_new = X_test[:3]

In [17]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 9s 5ms/step - loss: 0.6833 - accuracy: 0.8275 - val_loss: 0.3667 - val_accuracy: 0.8994
Epoch 2/5
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3480 - accuracy: 0.9028 - val_loss: 0.2966 - val_accuracy: 0.9174
Epoch 3/5
1719/1719 [==============================] - 9s 5ms/step - loss: 0.2982 - accuracy: 0.9161 - val_loss: 0.2618 - val_accuracy: 0.9286
Epoch 4/5
1719/1719 [==============================] - 9s 5ms/step - loss: 0.2678 - accuracy: 0.9250 - val_loss: 0.2403 - val_accuracy: 0.9336
Epoch 5/5
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2449 - accuracy: 0.9316 - val_loss: 0.2229 - val_accuracy: 0.9402


In [18]:
np.round(model.predict(X_new), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.01, 0.  , 0.97, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

In [19]:
model_version = "0001"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
model_path

'my_mnist_model\\0001'

In [20]:
tf.saved_model.save(model, model_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_mnist_model\0001\assets


In [21]:
for root, dirs, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

my_mnist_model/
    .ipynb_checkpoints/
    0001/
        saved_model.pb
        assets/
        variables/
            variables.data-00000-of-00001
            variables.index


In [23]:
!saved_model_cli show --dir {model_path}

The given SavedModel contains the following tag-sets:
'serve'


2020-06-13 21:34:27.126953: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-06-13 21:34:27.128074: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [25]:
!saved_model_cli show --dir {model_path} --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


2020-06-11 21:00:18.516966: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-06-11 21:00:18.517950: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [27]:
!saved_model_cli show --dir {model_path} --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28, 1)
      name: serving_default_flatten_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


2020-06-11 21:10:36.181679: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-06-11 21:10:36.182695: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [28]:
!saved_model_cli show --dir {model_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_flatten_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          flatte

2020-06-11 21:11:32.911033: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-06-11 21:11:32.912091: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [29]:
np.save("my_mnist_tests.npy", X_new)

In [30]:
input_name = model.input_names[0]
input_name

'flatten_input'

In [32]:
!saved_model_cli run --dir {model_path} --tag_set serve \
                     --signature_def serving_default    \
                     --inputs {input_name}=my_mnist_tests.npy

Result for output key dense_1:
[[2.5549411e-05 2.6388682e-07 5.4926169e-04 4.2934958e-03 1.1866863e-06
  5.1868155e-05 5.4799569e-09 9.9477661e-01 1.2827671e-05 2.8901084e-04]
 [1.9608217e-03 1.5206332e-04 9.8230988e-01 1.1362790e-02 4.0311674e-09
  1.5509254e-03 1.5148235e-03 1.0319420e-09 1.1487898e-03 8.3476213e-09]
 [3.6493067e-05 9.8435313e-01 5.0017419e-03 1.9346623e-03 4.4545042e-04
  1.5790232e-03 1.4210178e-03 2.5341371e-03 2.2951984e-03 3.9908598e-04]]


2020-06-11 21:17:05.664280: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-06-11 21:17:05.665671: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2020-06-11 21:17:16.718675: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library nvcuda.dll
2020-06-11 21:17:16.793410: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro M1000M computeCapability: 5.0
coreClock: 1.0715GHz coreCount: 4 deviceMemorySize: 2.00GiB deviceMemoryBandwidth: 74.65GiB/s
2020-06-11 21:17:16.797087: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-06-11 21:17:16.800032: W tensorflow/stream_executor/platform/d

In [7]:
import json

input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [8]:
repr(input_data_json)[:1500] + "..."

'\'{"signature_name": "serving_default", "instances": [[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0

In [9]:
model = tf.saved_model.load(model_path)

!docker run --rm -p 8500:8500 -p 8501:8501 \
   -v "C:\Users\user\Documents\AI\ML_and_Deep_Learning\my_mnist_model: /models/my_mnist_model" \
   -e MODEL_NAME=my_mnist_model \
   tensorflow/serving

## REST API

In [37]:
import requests

SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()

In [28]:
response.keys()

dict_keys(['predictions'])

In [30]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.01, 0.  , 0.97, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

## gRPC API

In [31]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.input_names[0]
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [32]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

In [33]:
response

outputs {
  key: "dense_3"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 3
      }
      dim {
        size: 10
      }
    }
    float_val: 6.273723556660116e-05
    float_val: 5.212639848650724e-07
    float_val: 0.0004618035163730383
    float_val: 0.0029847798869013786
    float_val: 6.387478606484365e-06
    float_val: 0.00014513287169393152
    float_val: 9.661780353553695e-08
    float_val: 0.99562007188797
    float_val: 3.742794069694355e-05
    float_val: 0.0006810689228586853
    float_val: 0.005194667726755142
    float_val: 8.643882028991356e-05
    float_val: 0.9741851687431335
    float_val: 0.008039184845983982
    float_val: 6.386037654237953e-08
    float_val: 0.002929819282144308
    float_val: 0.007848480716347694
    float_val: 3.3740487737077274e-08
    float_val: 0.0017157524125650525
    float_val: 5.001268164050998e-07
    float_val: 4.3590047425823286e-05
    float_val: 0.9665178060531616
    float_val: 0.010660591535270214
    flo

In [34]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.01, 0.  , 0.97, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

In [35]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
shape = [dim.size for dim in outputs_proto.tensor_shape.dim]
y_proba = np.array(outputs_proto.float_val).reshape(shape)
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.01, 0.  , 0.97, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

## Deploying New Version

In [38]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-2),
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.7039 - accuracy: 0.8056 - val_loss: 0.3418 - val_accuracy: 0.9042
Epoch 2/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3204 - accuracy: 0.9082 - val_loss: 0.2674 - val_accuracy: 0.9242
Epoch 3/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2650 - accuracy: 0.9235 - val_loss: 0.2227 - val_accuracy: 0.9366
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2318 - accuracy: 0.9330 - val_loss: 0.2032 - val_accuracy: 0.9432
Epoch 5/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2088 - accuracy: 0.9398 - val_loss: 0.1831 - val_accuracy: 0.9478
Epoch 6/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.1907 - accuracy: 0.9446 - val_loss: 0.1740 - val_accuracy: 0.9498
Epoch 7/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.1755 - accuracy: 0.9491 - val_loss: 0.1604 - val_accuracy:

In [39]:
model_version = "0002"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
model_path

'my_mnist_model\\0002'

In [40]:
tf.saved_model.save(model, model_path)

INFO:tensorflow:Assets written to: my_mnist_model\0002\assets


In [41]:
for root, dirs, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

my_mnist_model/
    .ipynb_checkpoints/
    0002/
        saved_model.pb
        assets/
        variables/
            variables.data-00000-of-00001
            variables.index
    1/
        saved_model.pb
        .ipynb_checkpoints/
        variables/
            variables.data-00000-of-00001
            variables.index


In [42]:
import requests

SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
            
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status()
response = response.json()

In [43]:
response.keys()

dict_keys(['predictions'])

In [44]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.99, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

# GPUs

In [20]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [45]:
tf.test.is_built_with_cuda()

True

In [46]:
from tensorflow.python.client.device_lib import list_local_devices

devices = list_local_devices()
devices

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3606110984841214198,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7269321735224843922
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 15646266680312922375
 physical_device_desc: "device: XLA_GPU device"]

In [47]:
def create_model():
    return keras.models.Sequential([
        keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                            padding="same", input_shape=[28, 28, 1]),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"), 
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(units=10, activation='softmax'),
    ])

In [49]:
batch_size = 100
model = create_model()
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=2,
          validation_data=(X_valid, y_valid), batch_size=batch_size)

Epoch 1/2
550/550 [==============================] - 69s 126ms/step - loss: 1.0959 - accuracy: 0.6788 - val_loss: 0.3690 - val_accuracy: 0.9002
Epoch 2/2
550/550 [==============================] - 69s 126ms/step - loss: 0.3708 - accuracy: 0.8894 - val_loss: 0.2426 - val_accuracy: 0.9320


In [50]:
distribution = tf.distribute.MirroredStrategy()

# Change the default all-reduce algorithm:
#distribution = tf.distribute.MirroredStrategy(
#    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

# Specify the list of GPUs to use:
#distribution = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

# Use the central storage strategy instead:
#distribution = tf.distribute.experimental.CentralStorageStrategy()

#resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
#tf.tpu.experimental.initialize_tpu_system(resolver)
#distribution = tf.distribute.experimental.TPUStrategy(resolver)

with distribution.scope():
    model = create_model()
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=keras.optimizers.SGD(lr=1e-2),
                  metrics=["accuracy"])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [52]:
batch_size = 100 # must be divisible by the number of workers
model.fit(X_train, y_train, epochs=2,
          validation_data=(X_valid, y_valid), batch_size=batch_size)

Epoch 1/2
550/550 [==============================] - 77s 140ms/step - accuracy: 0.7359 - loss: 0.9223 - val_accuracy: 0.9036 - val_loss: 0.3581
Epoch 2/2
550/550 [==============================] - 75s 137ms/step - accuracy: 0.8900 - loss: 0.3663 - val_accuracy: 0.9354 - val_loss: 0.2354


In [53]:
model.predict(X_new)

array([[3.7757377e-06, 1.3624029e-07, 2.0045125e-06, 1.7148317e-04,
        5.7572930e-07, 2.0462519e-06, 5.1779550e-09, 9.9930966e-01,
        4.7250678e-06, 5.0551631e-04],
       [1.4141438e-03, 2.3902810e-04, 9.7509980e-01, 2.5901517e-03,
        2.2458783e-08, 1.0474585e-03, 1.8372359e-02, 7.6962579e-09,
        1.2370666e-03, 1.3929133e-08],
       [1.2412974e-04, 9.8122448e-01, 3.2552867e-03, 2.4611687e-03,
        1.6168872e-03, 3.2785774e-04, 2.7601044e-03, 3.3921045e-03,
        4.1571539e-03, 6.8077416e-04]], dtype=float32)